In [1]:
import numpy as np

In [2]:
class Variety():
    def __init__(self, faces):
        self.faces = faces
        self.n = np.max(faces) + 1

        # Check correctness
        vertices = set(i for f in faces for i in f)
        assert set(range(self.n)) == vertices
        for f in faces:
            assert len(f)==3

    def check(self):
        self.adjacency = np.zeros((self.n, self.n), dtype=np.int32)
        faces = []

        # Check half-edges and faces
        for face in self.faces:
            if sorted(face) in faces:
                return False
            faces += [sorted(face)]
            self.adjacency[face[0]][face[1]] += 1
            self.adjacency[face[1]][face[2]] += 1
            self.adjacency[face[2]][face[0]] += 1
        if np.any(self.adjacency != self.adjacency.T) or len(np.where(self.adjacency > 1)[0]) > 0:
            return False

        # Check links
        for v in range(self.n):
            seen = []
            edges = []
            vertices = np.where(self.adjacency[v] > 0)[0]
            v_from = vertices[0]
            for face in self.faces:
                if v in face:
                    edges += [np.setdiff1d(face, [v])]
            while True:
                seen += [v_from]
                candidates = []
                v_to = -1
                for edge in edges:
                    if v_from in edge:
                        candidates += list(np.setdiff1d(edge, [v_from]))
                if len(candidates) != 2:
                    return False
                for candidate in candidates:
                    if not candidate in seen:
                        v_to = candidate
                if v_to == -1 and len(seen) == len(vertices):
                    break
                v_from = v_to

        return True

    def Euler(self):
        return len(self.faces) - int(np.sum(self.adjacency) / 2) + self.n

    def d_0(self, func):
        return lambda x, y: func(y) - func(x)

    def d_1(self, func):
        return lambda x, y, z: func(x, y) + func(y, z) + func(z, x)

    def check_form(self, k, func):
        if k == 1:
            edges = []
            for face in self.faces:
                edges += [(face[0], face[1]), (face[1], face[2]), (face[2], face[0])]
            for edge in edges:
                if func(edge[0], edge[1]) != -func(edge[1], edge[0]):
                    return False
        elif k == 2:
            for face in self.faces:
                x = face[0]
                y = face[1]
                z = face[2]
                if not ((func(x, y, z) == func(y, z, x) == func(z, x, y)) and
                        (func(x, z, y) == func(y, x, z) == func(z, y, x)) and
                        (func(x, y, z) == -func(x, z, y))):
                    return False
        return True

    def wedge(self, k1, k2, f1, f2):
        if k1 == 0 and k2 == 0:
            return lambda x: f1(x) * f2(x)
        if k1 == 0 and k2 == 1:
            return lambda x, y: (f1(x) + f1(y)) * f2(x, y) / 2
        if k1 == 1 and k2 == 0:
            return lambda x, y: (f2(x) + f2(y)) * f1(x, y) / 2
        if k1 == 0 and k2 == 2:
            return lambda x, y, z: (f1(x) + f1(y) + f1(z)) * f2(x, y, z) / 3
        if k1 == 2 and k2 == 0:
            return lambda x, y, z: (f2(x) + f2(y) + f2(z)) * f1(x, y, z) / 3
        if k1 == 1 and k2 == 1:
            return lambda x, y, z: (f1(x, y) * f2(y, z) - f1(y, z) * f2(x, y) +
                                    f1(y, z) * f2(z, x) - f1(z, x) * f2(y, z) +
                                    f1(z, x) * f2(x, y) - f1(x, y) * f2(z, x)) / 6

In [3]:
sphere = Variety([(3,2,1), (2,3,0), (1,0,3), (0,1,2)])
torus = Variety([(1,0,3), (1,3,2), (2,3,6), (3,4,6),
                 (4,0,6), (1,6,0), (2,6,5), (1,5,6),
                 (2,5,0), (3,0,5), (5,4,3), (1,4,5),
                 (1,2,4), (2,0,4)])

def test1():
    assert sphere.check() == True
    assert torus.check() == True
    assert Variety([(1,2,3), (2,3,0), (3,0,1), (0,1,2)]).check() == False
    assert Variety([(1,2,3), (2,3,1), (3,0,1), (0,1,2)]).check() == False
    assert Variety([(1,2,0), (1,0,2)]).check() == False
    assert Variety([(3,2,1), (2,3,0), (1,0,3), (0,1,2),
                    (6,5,4), (5,6,0), (4,0,6), (0,4,5)]).check() == False

def test2():
    assert sphere.Euler() == 2
    assert torus.Euler() == 0

def test3():
    assert sphere.d_0(lambda x: x)(1, 2) == 1
    assert torus.d_0(lambda x: x **2 )(4, 3) == -7

def test4():
    assert sphere.check_form(2, lambda x, y, z: x + y + z
                             if (x - y) * (y - z) * (z - x) > 0 else -(x + y + z)) == True
    assert torus.check_form(1, lambda v, w: 1) == False

def test5():
    assert sphere.d_1(lambda x, y: x - y)(0, 1, 2) == 0
    assert sphere.d_1(lambda x, y: x * y if x < y else -x * y)(1, 2, 3) == 5

def test61():
    assert sphere.wedge(0, 0, lambda x: x, lambda x: -x)(2) == -4

def test62():
    assert sphere.wedge(0, 1, lambda x: x, lambda x, y: y - x)(2, 3) == 5/2

def test63():
    assert sphere.wedge(0, 2, lambda x: x, lambda x, y, z:  x + y + z
                        if (x - y) * (y - z) * (z - x) > 0 else -(x + y + z) )(0, 1, 2) == 3

def test64():
    assert sphere.wedge(1, 1, lambda x, y: x - y, lambda x, y: y - x)(1, 2, 3) == 0

tests = [(1, test1),
         (2, test2),
         (3, test3),
         (4, test4),
         (5, test5),
         ("6.1", test61),
         ("6.2", test62),
         ("6.3", test63),
         ("6.4", test64)]

for i, t in tests:
    try:
        t()
    except:
        print('Функция задания {0} не прошла проверку'.format(i))
    else:
        print('Ошибок в задании {} не найдено'.format(i))

Ошибок в задании 1 не найдено
Ошибок в задании 2 не найдено
Ошибок в задании 3 не найдено
Ошибок в задании 4 не найдено
Ошибок в задании 5 не найдено
Ошибок в задании 6.1 не найдено
Ошибок в задании 6.2 не найдено
Ошибок в задании 6.3 не найдено
Ошибок в задании 6.4 не найдено
